In [1]:
from pyspark.sql import SparkSession

# -------------------
# Spark session
# -------------------
spark = (
    SparkSession.builder
        .appName("Delta S3 Hive Auto Insert")
        .master("local[*]")
        .config("spark.jars.packages",
                "io.delta:delta-spark_2.13:4.0.0,"
                "org.apache.hadoop:hadoop-aws:3.4.1,"
                "org.apache.spark:spark-avro_2.13:3.4.1")  # <- add this
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        # MinIO / S3
        .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
        .config("spark.hadoop.fs.s3a.access.key", "admin")
        .config("spark.hadoop.fs.s3a.secret.key", "password")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
        .config("spark.sql.warehouse.dir", "s3a://warehouse/")
        # Hive Metastore
        .config("spark.hadoop.hive.metastore.uris", "thrift://localhost:9083")
        .config("spark.sql.catalogImplementation", "hive")
        .getOrCreate()
)


# Set logging level to ERROR to reduce warnings
spark.sparkContext.setLogLevel("ERROR")

print("Spark session started with ERROR log level")

# Check current catalog
current_catalog = spark.catalog.currentCatalog()
print("Current catalog:", current_catalog)

# Check current database
current_db = spark.catalog.currentDatabase()
print("Current database:", current_db)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/04 17:40:46 WARN Utils: Your hostname, ubuntu2020-virtual-machine, resolves to a loopback address: 127.0.1.1; using 192.168.87.134 instead (on interface ens33)
26/01/04 17:40:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/ubuntu2020/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/ubuntu2020/.ivy2.5.2/cache
The jars for the packages stored in: /home/ubuntu2020/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.spark#spark-avro_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-69004aaa-6270-4368-9b2a-b8c3384fe18b;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in centr

Spark session started with ERROR log level


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


Current catalog: spark_catalog
Current database: default


In [4]:
# -------------------
# Correct FS copy: local -> S3A (MinIO)
# -------------------
sc = spark.sparkContext
hadoop_conf = sc._jsc.hadoopConfiguration()

Path = sc._jvm.org.apache.hadoop.fs.Path
FileSystem = sc._jvm.org.apache.hadoop.fs.FileSystem
FileUtil = sc._jvm.org.apache.hadoop.fs.FileUtil

src = "/home/ubuntu2020/dataset/01_MRI_Data/*"
dst = "s3a://warehouse/fs/mri/01_MRI_Data/"

src_path = Path(src)
dst_path = Path(dst)

# IMPORTANT: get filesystem from PATH, not default
src_fs = src_path.getFileSystem(hadoop_conf)   # file://
dst_fs = dst_path.getFileSystem(hadoop_conf)   # s3a://

# Ensure destination exists
if not dst_fs.exists(dst_path):
    dst_fs.mkdirs(dst_path)

# Recursive copy
FileUtil.copy(
    src_fs,
    src_path,
    dst_fs,
    dst_path,
    False,   # deleteSource = False
    hadoop_conf
)

print("Copy completed successfully: local to MinIO")

Copy completed successfully: local to MinIO


In [9]:
df = spark.read.format("binaryFile").option("recursiveFileLookup", "true").load("s3a://warehouse/fs/mri/01_MRI_Data/01_MRI_Data/").select("path", "length")

df.show(20, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|path                                                                                                                                                          |length|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|s3a://warehouse/fs/mri/01_MRI_Data/01_MRI_Data/0006/L-SPINE_LSS_20160329_160303_952000/POSDISP_[5]_T2_TSE_TRA_384_5001/POSDISP_[5]_0006_001.ima               |559938|
|s3a://warehouse/fs/mri/01_MRI_Data/01_MRI_Data/0008/L-SPINE_CLINICAL_LIBRARIES_20160605_112202_964000/POSDISP_[5]_T2_TSE_TRA_384_5001/POSDISP_[5]_0008_001.ima|559802|
|s3a://warehouse/fs/mri/01_MRI_Data/01_MRI_Data/0008/L-SPINE_CLINICAL_LIBRARIES_20160605_112202_964000/POSDISP_[5]_T2_TSE_TRA_384_5001/POSDISP_[5]_0008_002.ima|

In [6]:
spark.read.format("binaryFile") \
    .load("s3a://warehouse/fs/mri/01_MRI_Data/01_MRI_Data/0001") \
    .select("path", "length") \
    .show(5, truncate=False)

+----+------+
|path|length|
+----+------+
+----+------+

